In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# 카뎃 목록 지정
cadets = "suhshin dha"

In [3]:
# notion api key 가져오기
load_dotenv()
my_bearer = os.getenv("BEARER")

In [4]:
libraryDbUrl = "https://api.notion.com/v1/databases/94e689f5b19947b1ab0f9b5f2d52962b/query"
contributorDbUrl = "https://api.notion.com/v1/databases/10909fc406ab42c29fa7f82d6a08d741/query"

In [5]:
headers = {
    "Authorization": "Bearer {}".format(my_bearer),
    "Accept": "application/json",
    "Notion-Version": "2022-02-22",
    "Content-Type": "application/json"
}

In [6]:
libraryBody = {
    "filter": {
        "and": [
            {
                "property": "create_date",
                "created_time": {
                    "on_or_after": "2022-01-01",
                }
            },
            {
                "property": "visible",
                "select": {
                    "does_not_equal": "no"
                }
            }
        ]
    },
    "page_size": 100
}

In [7]:
contributorBody = {
    "filter": {
        "property": "구분",
        "multi_select" : {
            "contains": "2기"
        }
    }
}

In [8]:
contributorResponse = requests.request("POST", contributorDbUrl, headers=headers, data=json.dumps(contributorBody))
contributorObject = json.loads(contributorResponse.text)

In [9]:
print(contributorObject["results"][0]["properties"])

{'\x08email': {'id': 'Glaa', 'type': 'rich_text', 'rich_text': []}, '\x08노션계정': {'id': 'NPPd', 'type': 'people', 'people': []}, '구분': {'id': '%60%3DBq', 'type': 'multi_select', 'multi_select': [{'id': '106c3f1e-ed6a-4d93-9804-c09a04c7ea88', 'name': '2기', 'color': 'yellow'}]}, 'Library DB (속성)에 관계됨': {'id': 'olLW', 'type': 'relation', 'relation': []}, '이름': {'id': 'title', 'type': 'title', 'title': []}, '한 줄 소개': {'id': '272a91ea-fbf5-4fd5-bf7a-994d4618a040', 'type': 'rich_text', 'rich_text': []}, 'blog': {'id': '4b55e315-b829-4237-be71-a7133a610904', 'type': 'url', 'url': None}, 'Stack': {'id': '8b48633a-4f75-4592-ac91-632b8de21fa0', 'type': 'multi_select', 'multi_select': []}, 'github': {'id': 'd347af9d-caea-4a57-902d-6edf9d1b493a', 'type': 'url', 'url': None}}


In [10]:
contributorList = []
for contributor in contributorObject["results"]:
    contributorList.append(contributor["properties"]["이름"]['title'])
    print(contributor["properties"]["이름"]['title'])

[]
[{'type': 'text', 'text': {'content': 'sohlee', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'sohlee', 'href': None}]
[{'type': 'text', 'text': {'content': 'dohyukim', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'dohyukim', 'href': None}]
[{'type': 'text', 'text': {'content': 'mingkim', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'mingkim', 'href': None}]
[{'type': 'text', 'text': {'content': 'sumkim', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'sumkim', 'href': None}]
[{'type': 'text', 'text': {'content': 'go', 'link': None}, 'annotations

In [ ]:
print(contributorList)
print(len(contributorList))

In [ ]:
libraryResponse = requests.request("POST", libraryDbUrl, headers=headers, data=json.dumps(libraryBody))
libraryObject = json.loads(libraryResponse.text)
libraryObjectList = []
libraryObjectList.extend(libraryObject["results"])
print(len(libraryObjectList))

In [ ]:
print(libraryObjectList[0]["properties"]["visible"]["select"])

In [ ]:
print(libraryObject["next_cursor"])

In [ ]:
libraryBody["start_cursor"] = "c116ce03-2170-40d2-9706-dd9afa75e9f6"

In [ ]:
print(libraryBody)

In [ ]:

def filterByContributors(x, cadet):
    return cadet == x["properties"]["rich_text"]["formula"]["string"]

In [ ]:
resultList = []
for contributor in contributorList:
        result = [x for x in libraryObjectList if filterByContributors(x, contributor)]
        resultList.append([contributor, len(result)])

In [109]:
df = pd.DataFrame(resultList)
df.columns = ['contributor', 'yes']
df["yes"].astype(int)

0     0
1     0
2     2
3     2
4     0
     ..
67    0
68    3
69    0
70    0
71    0
Name: yes, Length: 72, dtype: int64

In [110]:
mask = (df.yes >= 2)

In [111]:
df_done = df.loc[mask,:]
df_done.sort_values(by='yes',axis=0, ascending=False)

,contributor,yes
10,jmaing,10
61,joonhan,7
30,sujpark,6
28,sangtale,3
40,yunselee,3
68,chanhale,3
2,mingkim,2
3,sumkim,2
6,minjkim2,2
16,woojlee,2
